In [1]:
from google.colab import drive

drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
folder_path = '/content/drive/My Drive/Indian Currency Dataset'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image
import numpy as np

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train = os.path.join(folder_path, 'train')
test = os.path.join(folder_path, 'test')

In [ ]:
train_fake = os.path.join(train, 'fake')
train_real = os.path.join(train, 'real')

In [ ]:
train_fake_images = [os.path.join(train_fake, img) for img in os.listdir(train_fake)]
train_real_images = [os.path.join(train_real, img) for img in os.listdir(train_real)]

In [ ]:
train_fake, val_fake = train_test_split(train_fake_images, test_size=0.15, random_state=42)
train_real, val_real = train_test_split(train_real_images, test_size=0.15, random_state=42)

In [ ]:
train_images = train_fake + train_real
val_images = val_fake + val_real

In [ ]:
validation = os.path.join(folder_path, 'validation')
os.makedirs(validation, exist_ok=True)

In [ ]:
val_fake = os.path.join(validation, 'fake')
val_real = os.path.join(validation, 'real')
os.makedirs(val_fake, exist_ok=True)
os.makedirs(val_real, exist_ok=True)

In [ ]:
for img_path in val_fake:
    shutil.move(img_path, os.path.join(val_fake, os.path.basename(img_path)))

In [ ]:
for img_path in val_real:
    shutil.move(img_path, os.path.join(val_real, os.path.basename(img_path)))

In [ ]:
print(f'Train set size: {len(train_images)}')
print(f'Validation set size: {len(val_images)}')

Train set size: 150
Validation set size: 27


In [ ]:
batch_size = 32
target_size = (224, 224)
train_generator = datagen.flow_from_directory(
    train,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_directory(
    validation,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=42
)

test_generator = datagen.flow_from_directory(
    test,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 150 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 107 images belonging to 2 classes.


In [ ]:
print(os.listdir(train))
print(os.listdir(validation))
print(os.listdir(test))

['fake', 'real']
['fake', 'real']
['fake', 'real']


In [ ]:
model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
model.trainable = False

In [ ]:
flatten_layer = tf.keras.layers.Flatten()(model.output)
dropout_layer = tf.keras.layers.Dropout(0.5)(flatten_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dropout_layer)

model = tf.keras.models.Model(model.input, output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
num_epochs = 5
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.5426 - loss: 2.1564 - val_accuracy: 0.8500 - val_loss: 0.4413
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - accuracy: 0.7980 - loss: 0.9740 - val_accuracy: 0.8500 - val_loss: 0.5116
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.8025 - loss: 0.6975 - val_accuracy: 0.8667 - val_loss: 0.4617
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.8695 - loss: 0.8211 - val_accuracy: 0.8500 - val_loss: 0.5401
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 4s/step - accuracy: 0.8363 - loss: 0.4478 - val_accuracy: 0.8333 - val_loss: 0.6877


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7996 - loss: 0.7412
Test Loss: 0.5099099278450012, Test Accuracy: 0.8504672646522522


In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator)
print(f'Train Loss: {train_loss}, Train Accuracy: {train_accuracy}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.9197 - loss: 0.2611
Train Loss: 0.3423157036304474, Train Accuracy: 0.8933333158493042


In [ ]:
predictions = model.predict(test_generator)

4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step


In [ ]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
def predict_currency(image_path, model):
    preprocessed_image = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    if prediction[0][0] >= 0.5:
       return "Real Currency"
    else:
       return "Fake Currency"

In [ ]:
image_path = '/content/drive/My Drive/Indian Currency Dataset/2000.jpg'
result = predict_currency(image_path, model)
print(f"The predicted result is: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
The predicted result is: Real Currency


In [ ]:
image_path = '/content/drive/My Drive/Indian Currency Dataset/test (52).jpg'
result = predict_currency(image_path, model)
print(f"The predicted result is: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
The predicted result is: Fake Currency
